In [53]:
# download VGG19 model
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn = models.vgg19_bn(pretrained=True, progress=True).features.to(device).eval()

In [54]:
# check model architecture
print(cnn)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [60]:
# prepare data and images

# imsize = 512 if torch.cuda.is_available() else 128  # use small size if no gpu
imsize = 512 
loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor
def image_loader(image_name):
    image = Image.open(image_name)
    print(image.size)
    print(image)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    # return image to torch tensor
    return image.to(device, torch.float)


img_1 = image_loader("../data/MoviePosters/3.jpg")
img_2 = image_loader("../data/MoviePosters/5.jpg")

print(img_1.shape)
print(img_2.shape)

if img_1.shape == img_2.shape:
    print('same size')
else:
    print('different size')


(182, 268)
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=182x268 at 0x2880BEBB0>
(182, 268)
<PIL.JpegImagePlugin.JpegImageFile image mode=L size=182x268 at 0x151747580>
torch.Size([1, 3, 753, 512])
torch.Size([1, 1, 753, 512])
different size
